In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.style
matplotlib.style.use("seaborn")
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import MinMaxScaler
import seaborn as sn
sn.color_palette("hls", 17)
import scipy.stats as st
import math

from pingouin import rcorr
import pingouin as pg

In [2]:
import sys
import os
sys.path.append(os.path.join(".."))
              
from module.trend_analysis_functions import *
from module.topic_model_plus_class import Topic_Model_plus

In [3]:
incident_file = os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_model_summaryreps.csv')
incident_summary_df = pd.read_csv(incident_file)
incident_summary_df = incident_summary_df.drop("Unnamed: 0", axis=1)
incident_summary_df = incident_summary_df.loc[incident_summary_df["START_YEAR"]>=2006].reset_index(drop=True)

C:\Users\srandrad\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3050: DtypeWarning: Columns (7,13,18,19,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Hazard Metrics

In [4]:
document_id_col = "INCIDENT_ID"
extra_cols = ["CY","DISCOVERY_DATE", "START_YEAR", "REPORT_DOY", "DISCOVERY_DOY",
              "TOTAL_PERSONNEL", "TOTAL_AERIAL", "PCT_CONTAINED_COMPLETED"]
list_of_attributes = ["Combined Text"]
file = os.path.join(os.path.dirname(os.getcwd()),'data', 'ICS_filtered_preprocessed_data_extra_cols.csv')
ICS = Topic_Model_plus(document_id_col=document_id_col, extra_cols=extra_cols, list_of_attributes=list_of_attributes, combine_cols=False)
ICS.extract_preprocessed_data(file)

C:\Users\srandrad\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3242: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
preprocessed_df = ICS.data_df
hazard_file = os.path.join(os.path.dirname(os.getcwd()),'results','hazard_interpretation_v2.xlsx')

In [6]:
preprocessed_df

,CY,DISCOVERY_DATE,INCIDENT_ID,PCT_CONTAINED_COMPLETED,START_YEAR,TOTAL_AERIAL,TOTAL_PERSONNEL,REPORT_DOY,DISCOVERY_DOY,Combined Text,...,EST_IM_COST_TO_DATE,STR_DAMAGED,STR_DESTROYED,NEW_ACRES,POO_STATE,POO_LATITUDE,POO_LONGITUDE,WEATHER_CONCERNS_NARR,INC_MGMT_ORG_ABBREV,EVACUATION_IN_PROGRESS
0,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,80.0,2010.0,5.000000,230.000000,197,196,"[resource, share, cactus]",...,10000.0,0.0,0.0,70.0,CA,33.619444,-116.900000,Current Weather RH: 45 TEMP: 75 WS: 10 WD: VAR...,4,False
1,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,5.000000,230.000000,197,196,"[resource, share, incident, cactus, incident, ...",...,90000.0,0.0,0.0,503.0,CA,33.666667,-117.000000,Current Weather RH: 52 TEMP: 78 WS: 12 WD: Var...,4,False
2,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,30.0,2010.0,4.000000,165.000000,197,196,"[resource, share, cactus, erratic, wind, due, ...",...,45000.0,0.0,0.0,450.0,CA,33.650000,-116.900000,Current Weather RH: 52 TEMP: 78 WS: 12 WD: VAR...,4,False
3,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,100.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,10000.0,0.0,0.0,10.0,CA,33.619444,-116.900000,Current Weather RH: 21 TEMP: 102 WS: 12 WD: SW...,4,False
4,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,50000.0,0.0,0.0,0.0,CA,33.666667,-117.000000,Current Weather RH: 22 TEMP: 104 WS: 15 WD: SW,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44358,2014,2014-03-15 14:30:00,2014_VAVAS1403037_BEAVER LODGE RD.,100.0,2014.0,0.000000,13.000000,74,74,"[fast, spread, field]",...,450.0,1.0,1.0,0.0,VA,38.521389,-77.576111,NaN,NaN,NaN
44359,2014,2014-03-19 14:00:00,2014_VAVAS1406037_AIRPORT MOUNTAIN,85.0,2014.0,0.000000,18.500000,80,78,"[heavy, plume, primary, carrier]",...,4000.0,0.0,0.0,85.0,VA,37.243056,-82.103333,NaN,NaN,NaN
44360,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,0.0,2014.0,1.000000,95.000000,234,232,"[heavy, canyon, river, mainly, canyon, come, e...",...,50000.0,0.0,0.0,1900.0,WA,46.000395,-120.024213,imt has spot weather forecast for next 24 hour...,NaN,NaN
44361,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,86.0,2014.0,1.000000,120.000000,235,232,"[laid, night, test, wind, remain, canyon, peri...",...,185000.0,0.0,0.0,0.0,WA,46.000395,-120.024213,imt has spot weather forecast for next 24 hour...,NaN,NaN


In [7]:
time_of_occurence_days, time_of_occurence_pct_contained, frequency, fires, frequency_fires, categories, hazards, years, ids = calc_metrics(hazard_file, preprocessed_df, rm_outliers=True,unique_ids_col='Unique IDs')

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [04:34<00:00, 30.55s/it]


# Data Preparation
to prepare the data for the models, we need to add new columns including: 
- Region - categorical
- Hazard Occurence - one hot
- Severity - continuous
- Days Burning - continuous
- Reports so far - continuous
- total text from all reports

#### Region

In [8]:
regions = {'AICC':['AK'],
          'EACC':['MN', 'IA', 'WI', 'IL', 'IN', 'MO', 'MI', 'OH', 'WV', 'PA',
                 'NY', 'VT', 'NH', 'ME', 'MA', 'RI', 'CT', 'NJ', 'DE', 'MD'],
          'GBCC':['NV', 'UT', {'ID':["46n"]}],
           'SACC': [{'TX':["100w"]}, 'OK', 'AR', 'LA', 'MS', 'AL', 'GA', 'FL', 'SC', 'NC',
                   'TN', 'KY', 'VA'],
           'NWCC': ['OR','WA'],
           'CA':['CA'], #OSCC and ONCC about 38 N
           'NRCC':['MT', 'ND', {'ID':["46n"]}],
           'RMCC':['CO', 'WY', 'SD', 'KS', 'NE'],
           'SWCC':['NM', 'AZ', {'TX':["100w"]}],
           'HICC':['HI']
          }
states = [state for region in regions for state in regions[region] if not isinstance(state,dict)]
region_by_state = {state: region for state in states for region in regions if state in regions[region]}
#region_by_state

In [9]:
split_states = {'TX':[("100",'w'), ('less','SACC'), ('greater', 'SWCC')],
                'ID':[("46","n"), ('less', 'GBCC'), ('greater', 'NRCC')]}
direction_dict = {'n':'POO_LATITUDE', 's':'POO_LATITUDE', 'w':'POO_LONGITUDE', 'e':'POO_LONGITUDE'}

In [10]:
region_per_incident = []#{}
for i in range(len(incident_summary_df)):
    incident_region = None
    id_ = incident_summary_df.iloc[i]['INCIDENT_ID']
    state = incident_summary_df.iloc[i]['POO_STATE']
    if isinstance(state, float):
    #    print(id_)
        id_ = id_.split("_")
        if len(id_)>1:
            state = id_[1].split("-")[0]
            if len(state)>2:
                state = 'CA'#only one instance of this manually checked and verified it is in california
                print(id_)
    if state in region_by_state:
        incident_region = region_by_state[state]
    elif state in split_states:
        (pos, direct) = split_states[state][0]
        incident_loc = float(incident_summary_df.iloc[i][direction_dict[direct]])
        if incident_loc <= float(pos):
            incident_region = split_states[state][1][1]
        else:
            incident_region = split_states[state][2][1]
   
    region_per_incident.append(incident_region)

incident_summary_df['Incident_region'] = region_per_incident

['2012', '004462', 'COPCO']


In [11]:
sitrep_regions = []
for i in range(len(preprocessed_df)):
    rep_region = None
    id_ = preprocessed_df.iloc[i]['INCIDENT_ID']
    incident_df = incident_summary_df.loc[incident_summary_df['INCIDENT_ID']==id_].reset_index(drop=True)
    if len(incident_df)>1: 
        print("multiple summary reports")
    else:
        rep_region = incident_df.at[0,'Incident_region']
    sitrep_regions.append(rep_region)
preprocessed_df['Incident_region'] = sitrep_regions
#preprocessed_df

#### Number of Reports for incident so far & Days Burning

In [12]:
#add col for num of reports so far and Days Burning
reports_so_far = []
days_burning_so_far = []
for i in range(len(preprocessed_df)):
    unique_id = preprocessed_df.iloc[i]['Unique IDs'].split("_")[-1]
    reports_so_far.append(int(unique_id))
    #correct dates
    time_of_hazard = int(preprocessed_df.iloc[i]['REPORT_DOY'])
    start_date = int(preprocessed_df.iloc[i]['DISCOVERY_DOY'])
    if time_of_hazard<start_date: 
        #print("dates corrected")
        if time_of_hazard<30 and start_date<330: #report day is days since start, not doy 
            time_of_hazard+=start_date
        elif time_of_hazard<30 and start_date>=330:
            start_date = start_date-365 #fire spans two years
        else: #start and report day were incorrectly switched
            temp_start = start_date
            start_date = time_of_hazard
            time_of_hazard = temp_start
    days_burning_so_far.append(time_of_hazard-int(start_date))
preprocessed_df['NUM_REPORTS'] = reports_so_far
preprocessed_df['DAYS_BURING'] = days_burning_so_far

#### Severity and Hazard Occurrence for sit reps

In [13]:
total_severity = []
for i in range(len(preprocessed_df)):
    severity = int(preprocessed_df.iloc[i]['FATALITIES']) + int(preprocessed_df.iloc[i]['INJURIES']) + int(preprocessed_df.iloc[i]['STR_DAMAGED']) +  int(preprocessed_df.iloc[i]['STR_DESTROYED'])
    total_severity.append(severity)
preprocessed_df['Severity'] = total_severity

In [14]:
ids_per_hazard = {hazard:[fire for year in ids[hazard] for fire in ids[hazard][year]] for hazard in ids}
hazards = [hazard for hazard in ids_per_hazard]
hazard_df_data = {hazard:[] for hazard in hazards}

for hazard in hazards:
    for i in range(len(preprocessed_df)):
        target = 0 
        if preprocessed_df.iloc[i]['Unique IDs'] in ids_per_hazard[hazard.replace("_"," ")]:
            target = 1
        hazard_df_data[hazard].append(target)
for hazard in ids_per_hazard:
    preprocessed_df[hazard] = hazard_df_data[hazard]

In [15]:
hazards = [hazard for hazard in ids_per_hazard]
hazard_df_data = {hazard:[] for hazard in hazards}
for i in range(len(preprocessed_df)):
    incident_id = preprocessed_df.iloc[i]['INCIDENT_ID']
    num = preprocessed_df.iloc[i]['NUM_REPORTS']
    future_incident_df = preprocessed_df.loc[(preprocessed_df['INCIDENT_ID']==incident_id) & (preprocessed_df['NUM_REPORTS']>num)]
    hazard_occurence = [haz for haz in hazards if 1 in future_incident_df[haz].tolist()]
    for hazard in hazards:
        if hazard in hazard_occurence:
            hazard_df_data[hazard].append(1)
        else:
            hazard_df_data[hazard].append(0)
for hazard in ids_per_hazard:
    preprocessed_df[hazard] = hazard_df_data[hazard]

#### Severity and Hazard occurence for summary reports

In [16]:
fires_per_hazard = {hazard:[fire for year in fires[hazard] for fire in fires[hazard][year]] for hazard in fires}
incident_ids = incident_summary_df['INCIDENT_ID'].tolist()
hazards = [hazard.replace(" ", "_") for hazard in fires_per_hazard]
hazard_df_data = {hazard:[] for hazard in hazards}
for hazard in hazards:
    for fire_id in incident_ids:
        target = 0
        if fire_id in fires_per_hazard[hazard.replace("_"," ")]:
            target = 1
        hazard_df_data[hazard].append(target)
for hazard in hazards:
    incident_summary_df[hazard] = hazard_df_data[hazard]
total_severity = []
for i in range(len(incident_summary_df)):
    severity = int(incident_summary_df.iloc[i]['FATALITIES']) + int(incident_summary_df.iloc[i]['INJURIES_TOTAL']) + int(incident_summary_df.iloc[i]['STR_DAMAGED_TOTAL']) +  int(incident_summary_df.iloc[i]['STR_DESTROYED_TOTAL'])
    total_severity.append(severity)
incident_summary_df['Severity'] = total_severity

#### Total Text from incident reports so far

In [17]:
total_text_so_far = []
preprocessed_df['Total Incident Text'] = [[] for i in range(len(preprocessed_df))]
prev_id = 0 
for i in tqdm(range(len(preprocessed_df))):
    incident_id = preprocessed_df.iloc[i]['INCIDENT_ID']
    if prev_id == incident_id:
        preprocessed_df.at[i,'Total Incident Text'] = preprocessed_df.iloc[i-1]['Total Incident Text'] + preprocessed_df.iloc[i]['Combined Text']
    else:
        preprocessed_df.at[i,'Total Incident Text'] = preprocessed_df.iloc[i]['Combined Text']
    prev_id = incident_id


100%|██████████████████████████████████████████████████████████████████████████| 44363/44363 [00:13<00:00, 3172.58it/s]


In [18]:
preprocessed_df

,CY,DISCOVERY_DATE,INCIDENT_ID,PCT_CONTAINED_COMPLETED,START_YEAR,TOTAL_AERIAL,TOTAL_PERSONNEL,REPORT_DOY,DISCOVERY_DOY,Combined Text,...,Livestock,Law Violations,Military Base,Infrastructure,Extreme Weather,Ecological,Hazardous Terrain,Floods,Dry Weather,Total Incident Text
0,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,80.0,2010.0,5.000000,230.000000,197,196,"[resource, share, cactus]",...,0,0,0,0,1,0,1,0,0,"[resource, share, cactus]"
1,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,5.000000,230.000000,197,196,"[resource, share, incident, cactus, incident, ...",...,0,0,0,0,1,0,1,0,0,"[resource, share, cactus, resource, share, inc..."
2,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,30.0,2010.0,4.000000,165.000000,197,196,"[resource, share, cactus, erratic, wind, due, ...",...,0,0,0,0,1,0,1,0,0,"[resource, share, cactus, resource, share, inc..."
3,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,100.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0,0,0,0,1,0,1,0,0,"[resource, share, cactus, resource, share, inc..."
4,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0,0,0,0,1,0,1,0,0,"[resource, share, cactus, resource, share, inc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44358,2014,2014-03-15 14:30:00,2014_VAVAS1403037_BEAVER LODGE RD.,100.0,2014.0,0.000000,13.000000,74,74,"[fast, spread, field]",...,0,0,0,0,0,0,0,0,0,"[fast, spread, field]"
44359,2014,2014-03-19 14:00:00,2014_VAVAS1406037_AIRPORT MOUNTAIN,85.0,2014.0,0.000000,18.500000,80,78,"[heavy, plume, primary, carrier]",...,0,0,0,0,0,0,0,0,0,"[heavy, plume, primary, carrier]"
44360,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,0.0,2014.0,1.000000,95.000000,234,232,"[heavy, canyon, river, mainly, canyon, come, e...",...,0,0,0,0,0,0,0,0,0,"[heavy, canyon, river, mainly, canyon, come, e..."
44361,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,86.0,2014.0,1.000000,120.000000,235,232,"[laid, night, test, wind, remain, canyon, peri...",...,0,0,0,0,0,0,0,0,0,"[heavy, canyon, river, mainly, canyon, come, e..."


In [19]:
incident_summary_df.columns = incident_summary_df.columns.str.replace(" ", "_")
preprocessed_df.columns = preprocessed_df.columns.str.replace(" ","_")

# Cleaning and nan handling

In [20]:
preprocessed_df.isna().sum()

CY                            0
DISCOVERY_DATE                0
INCIDENT_ID                   0
PCT_CONTAINED_COMPLETED       0
START_YEAR                    0
TOTAL_AERIAL                  0
TOTAL_PERSONNEL               0
REPORT_DOY                    0
DISCOVERY_DOY                 0
Combined_Text                 0
Unique_IDs                    0
ACRES                         0
WF_FSR                        0
INJURIES                      0
FATALITIES                    0
EST_IM_COST_TO_DATE        7831
STR_DAMAGED                   0
STR_DESTROYED                 0
NEW_ACRES                     0
POO_STATE                  4481
POO_LATITUDE                101
POO_LONGITUDE               101
WEATHER_CONCERNS_NARR      5916
INC_MGMT_ORG_ABBREV        6882
EVACUATION_IN_PROGRESS     3294
Incident_region              20
NUM_REPORTS                   0
DAYS_BURING                   0
Severity                      0
Traffic                       0
Command_Transitions           0
Evacuati

In [21]:
#incident type
preprocessed_df['INC_MGMT_ORG_ABBREV'].unique()

array(['4', '3', nan, '2', '1', 'F', '5', 'B', 'C', 'D', '6', 'A', 'E',
       '8', '7'], dtype=object)

In [22]:
preprocessed_df['INC_MGMT_ORG_ABBREV'].value_counts()

2    8884
3    7104
4    6386
5    6377
1    4498
D    2237
6     894
F     455
8     222
B     133
C     111
E      89
7      62
A      29
Name: INC_MGMT_ORG_ABBREV, dtype: int64

In [23]:
preprocessed_df['INC_MGMT_ORG_ABBREV'].replace(to_replace={'6':'5','7':'1','8':'2',5:'5'}, inplace=True)

In [24]:
preprocessed_df['INC_MGMT_ORG_ABBREV'].value_counts()
#F -> FUMT
#A -> FUM1
#B -> FUM2
#C -> Area command
#D -> Unifed Command
#E -> SOPL
#replace nans based on training/test

2    9106
5    7271
3    7104
4    6386
1    4560
D    2237
F     455
B     133
C     111
E      89
A      29
Name: INC_MGMT_ORG_ABBREV, dtype: int64

In [25]:
#Replaced missing incident types with 5, lowest response type
preprocessed_df['INC_MGMT_ORG_ABBREV'].fillna('5', inplace=True)
#Removed the 20 reports with no region in the FS regions – these incidents were all in Puerto rico/US territories 
preprocessed_df = preprocessed_df[preprocessed_df['Incident_region'].notna()].reset_index(drop=True)
#Reports missing evacuation Boolean set to false 
preprocessed_df['EVACUATION_IN_PROGRESS'].fillna(False, inplace=True)
#Reports missing cost set the value to 0  
preprocessed_df['EST_IM_COST_TO_DATE'].fillna(0, inplace=True)

In [26]:
#Preparing regression model training set
hazard_model_set = preprocessed_df#pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'data','Cleaned_ICS_predictive_model_sitreps.csv'), dtype=str)
summary_reports = incident_summary_df

### Severity calculations
 - final severity for each metric
 - current severity for each metric
 - diff = final - current for each metric

In [27]:
final_injuries = []; current_injuries = []; diff_injuries = []
final_str_dam = []; current_str_dam = []; diff_str_dam = []
final_str_des = []; current_str_des = []; diff_str_des = []
final_fatalities = []; current_fatalities = []; diff_fatalities = []
severity_regression_model = preprocessed_df
curr_id = 0; curr_vals = {"injuries":0, "str_dam":0, "str_des":0, "fatalities":0}
i=0
for id_ in hazard_model_set['INCIDENT_ID']:
    if curr_id != id_: 
        curr_vals = {"injuries":0, "str_dam":0, "str_des":0, "fatalities":0}
        curr_id = id_
    summary_incident = summary_reports.loc[summary_reports['INCIDENT_ID']==id_].reset_index(drop=True)
    if len(summary_incident)>1:
        print(id_)
    final_injuries.append(summary_incident.at[0,'INJURIES_TOTAL'])
    final_str_dam.append(summary_incident.at[0,'STR_DAMAGED_TOTAL'])
    final_str_des.append(summary_incident.at[0,'STR_DESTROYED_TOTAL'])
    final_fatalities.append(summary_incident.at[0,'FATALITIES'])
    curr_vals['injuries'] = float(hazard_model_set.iloc[i]["INJURIES"]); current_injuries.append(curr_vals['injuries'])
    curr_vals['str_dam'] = float(hazard_model_set.iloc[i]['STR_DAMAGED']); current_str_dam.append(curr_vals['str_dam'])
    curr_vals['str_des'] = float(hazard_model_set.iloc[i]['STR_DESTROYED']); current_str_des.append(curr_vals['str_des'])
    curr_vals['fatalities'] = float(hazard_model_set.iloc[i]['FATALITIES']); current_fatalities.append(curr_vals['fatalities'])
    diff_injuries.append(float(summary_incident.at[0,'INJURIES_TOTAL']) - curr_vals['injuries'])
    diff_str_dam.append(float(summary_incident.at[0,'STR_DAMAGED_TOTAL']) - curr_vals['str_dam'])
    diff_str_des.append(float(summary_incident.at[0,'STR_DESTROYED_TOTAL']) - curr_vals['str_des'])
    diff_fatalities.append(float(summary_incident.at[0,'FATALITIES']) - curr_vals['fatalities'])
    i+=1
severity_regression_model['Current_total_Injuries'] = current_injuries
severity_regression_model['Current_total_Structures_Damages'] = current_str_dam
severity_regression_model['Current_total_Structures_Destroyed'] = current_str_des
severity_regression_model['Current_total_Fatalities'] = current_fatalities
severity_regression_model['Diff_Injuries'] = diff_injuries
severity_regression_model['Diff_Structures_Damages'] = diff_str_dam
severity_regression_model['Diff_Structures_Destroyed'] = diff_str_des
severity_regression_model['Diff_Fatalities'] = diff_fatalities
severity_regression_model['Total_Injuries'] = final_injuries
severity_regression_model['Total_Structures_Damages'] = final_str_dam
severity_regression_model['Total_Structures_Destroyed'] = final_str_des
severity_regression_model['Total_Fatalities'] = final_fatalities
severity_regression_model

,CY,DISCOVERY_DATE,INCIDENT_ID,PCT_CONTAINED_COMPLETED,START_YEAR,TOTAL_AERIAL,TOTAL_PERSONNEL,REPORT_DOY,DISCOVERY_DOY,Combined_Text,...,Current_total_Structures_Destroyed,Current_total_Fatalities,Diff_Injuries,Diff_Structures_Damages,Diff_Structures_Destroyed,Diff_Fatalities,Total_Injuries,Total_Structures_Damages,Total_Structures_Destroyed,Total_Fatalities
0,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,80.0,2010.0,5.000000,230.000000,197,196,"[resource, share, cactus]",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,5.000000,230.000000,197,196,"[resource, share, incident, cactus, incident, ...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,30.0,2010.0,4.000000,165.000000,197,196,"[resource, share, cactus, erratic, wind, due, ...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,100.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44338,2014,2014-03-15 14:30:00,2014_VAVAS1403037_BEAVER LODGE RD.,100.0,2014.0,0.000000,13.000000,74,74,"[fast, spread, field]",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
44339,2014,2014-03-19 14:00:00,2014_VAVAS1406037_AIRPORT MOUNTAIN,85.0,2014.0,0.000000,18.500000,80,78,"[heavy, plume, primary, carrier]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44340,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,0.0,2014.0,1.000000,95.000000,234,232,"[heavy, canyon, river, mainly, canyon, come, e...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44341,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,86.0,2014.0,1.000000,120.000000,235,232,"[laid, night, test, wind, remain, canyon, peri...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Fixing severity discrepancies

In [28]:
#identifying documents where the expected number of consequences is negative (summary val < sum situation vals)
incorrect_vals = {col:[val for val in severity_regression_model[col].unique() if val<0] for col in ['Diff_Injuries','Diff_Structures_Damages','Diff_Structures_Destroyed','Diff_Fatalities']}
ids_with_issues = []
for col in incorrect_vals:
    ids = severity_regression_model['INCIDENT_ID'].loc[severity_regression_model[col].isin(incorrect_vals[col])]
    for id_ in ids: 
        ids_with_issues.append(id_)
    #print(col, ids)
ids_with_issues = list(set(ids_with_issues))

In [29]:
#identifying documents where summary val != sum situation val
data = severity_regression_model
summary_data = incident_summary_df

sit_metrics = ['INJURIES','FATALITIES', 'STR_DESTROYED','STR_DAMAGED']
sum_metrics = ['INJURIES_TOTAL', 'FATALITIES', 'STR_DESTROYED_TOTAL','STR_DAMAGED_TOTAL']
dfs = {}
for i in range(len(sit_metrics)):
    ids = []
    from_situation = []
    from_summary = []
    for id_ in set(data["INCIDENT_ID"]):
        id_df = data.loc[data["INCIDENT_ID"]==id_]
        injuries_from_df = sum(id_df[sit_metrics[i]])
        injuries_from_summary = summary_data[sum_metrics[i]].loc[summary_data["INCIDENT_ID"]==id_]
        if injuries_from_df != int(injuries_from_summary):
            ids.append(id_)
            from_situation.append(injuries_from_df)
            from_summary.append(int(injuries_from_summary))
    dfs[sit_metrics[i]] = pd.DataFrame({'incident id':ids,
                                        'situation report sum': from_situation,
                                        'summary report sum': from_summary})
with pd.ExcelWriter(os.path.join(os.path.dirname(os.getcwd()),'data','incidents_with_severity_discrepancies.xlsx')) as writer2:
    for metric in dfs:
        dfs[metric].to_excel(writer2, sheet_name = metric, index = False)

incidents_to_drop = []
for metric in dfs:
    metric_df = dfs[metric]
    for id_ in metric_df['incident id']:
        incidents_to_drop.append(id_)
incidents_to_drop = list(set(incidents_to_drop))

In [30]:
incidents_to_drop += ids_with_issues

In [31]:
severity_regression_model = severity_regression_model.loc[~severity_regression_model['INCIDENT_ID'].isin(incidents_to_drop)].reset_index(drop=True)
severity_regression_model

,CY,DISCOVERY_DATE,INCIDENT_ID,PCT_CONTAINED_COMPLETED,START_YEAR,TOTAL_AERIAL,TOTAL_PERSONNEL,REPORT_DOY,DISCOVERY_DOY,Combined_Text,...,Current_total_Structures_Destroyed,Current_total_Fatalities,Diff_Injuries,Diff_Structures_Damages,Diff_Structures_Destroyed,Diff_Fatalities,Total_Injuries,Total_Structures_Damages,Total_Structures_Destroyed,Total_Fatalities
0,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,80.0,2010.0,5.000000,230.000000,197,196,"[resource, share, cactus]",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,5.000000,230.000000,197,196,"[resource, share, incident, cactus, incident, ...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,30.0,2010.0,4.000000,165.000000,197,196,"[resource, share, cactus, erratic, wind, due, ...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,100.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2010,2010-07-15 15:00:00,2000_CA-RRU-062485_VALLEY COMPLEX,60.0,2010.0,4.333333,192.333333,197,196,"[resource, share, cactus, cactus, become, vall...",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23323,2014,2014-03-15 14:30:00,2014_VAVAS1403037_BEAVER LODGE RD.,100.0,2014.0,0.000000,13.000000,74,74,"[fast, spread, field]",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
23324,2014,2014-03-19 14:00:00,2014_VAVAS1406037_AIRPORT MOUNTAIN,85.0,2014.0,0.000000,18.500000,80,78,"[heavy, plume, primary, carrier]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23325,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,0.0,2014.0,1.000000,95.000000,234,232,"[heavy, canyon, river, mainly, canyon, come, e...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23326,2014,2014-08-20 13:00:00,2014_WA-WFS-513_SAND RIDGE,86.0,2014.0,1.000000,120.000000,235,232,"[laid, night, test, wind, remain, canyon, peri...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
severity_regression_model.isna().sum()

CY                                       0
DISCOVERY_DATE                           0
INCIDENT_ID                              0
PCT_CONTAINED_COMPLETED                  0
START_YEAR                               0
TOTAL_AERIAL                             0
TOTAL_PERSONNEL                          0
REPORT_DOY                               0
DISCOVERY_DOY                            0
Combined_Text                            0
Unique_IDs                               0
ACRES                                    0
WF_FSR                                   0
INJURIES                                 0
FATALITIES                               0
EST_IM_COST_TO_DATE                      0
STR_DAMAGED                              0
STR_DESTROYED                            0
NEW_ACRES                                0
POO_STATE                             2084
POO_LATITUDE                            46
POO_LONGITUDE                           46
WEATHER_CONCERNS_NARR                 4281
INC_MGMT_OR

In [33]:
incident_summary_df = incident_summary_df.loc[~incident_summary_df['INCIDENT_ID'].isin(incidents_to_drop)].reset_index(drop=True)
incident_summary_df

,INCIDENT_ID,INCIDENT_NUMBER,INCIDENT_NAME,INCTYP_ABBREVIATION,FINAL_ACRES,CAUSE,COMPLEX,DISCOVERY_DATE,DISCOVERY_DOY,EXPECTED_CONTAINMENT_DATE,...,Livestock,Law_Violations,Military_Base,Infrastructure,Extreme_Weather,Ecological,Hazardous_Terrain,Floods,Dry_Weather,Severity
0,2000_CA-RRU-062485_VALLEY COMPLEX,CA-RRU-062485,VALLEY COMPLEX,WF,1312.0,L,True,2010-07-15 15:00:00,196,2010-07-18 00:00:00,...,0,0,0,0,1,0,1,0,0,1
1,2006_00276_MILLER COMPLEX,00276,MILLER COMPLEX,WF,300.0,L,True,2006-06-26 19:55:00,177,2006-07-03 18:00:00,...,0,0,0,0,0,1,0,0,0,0
2,2006_1224_MULTIPLE JUNE FIRES,1224,MULTIPLE JUNE FIRES,WF,21.0,L,False,2006-06-26 15:00:00,177,2006-07-01 18:00:00,...,0,0,0,0,0,0,1,0,0,0
3,2006_AK-DAS-612166_JARVIS CREEK,AK-DAS-612166,JARVIS CREEK,WF,275.0,H,False,2006-06-08 11:13:00,159,2006-06-13 23:59:00,...,0,0,0,0,0,0,0,0,0,0
4,2006_AK-FAS-611307_TEKLANIKA,AK-FAS-611307,TEKLANIKA,WF,142.0,L,False,2006-07-05 18:40:00,186,2006-08-07 11:00:00,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8082,2014_VAVAS1406037_AIRPORT MOUNTAIN,VAVAS1406037,Airport Mountain,WF,200.0,H,False,2014-03-19 14:00:00,78,NaN,...,0,0,0,0,0,0,0,0,0,0
8083,2014_VAVAS1406040_BAKERS RIDGE,VAVAS1406040,Bakers Ridge,WF,100.0,H,False,2014-03-22 12:00:00,81,NaN,...,0,0,0,0,0,0,0,0,0,0
8084,2014_VAVAS1406041_NEIGHBOR TRAIL,VAVAS1406041,Neighbor Trail,WF,180.0,H,False,2014-03-21 12:00:00,80,NaN,...,0,0,0,0,0,0,0,0,0,0
8085,2014_VAVAS1406043_PINE CREEK,VAVAS1406043,Pine Creek,WF,150.0,H,False,2014-03-21 13:00:00,80,NaN,...,0,0,0,0,0,0,0,0,0,0


### Building train, test, validation sets; saving data files

In [34]:
severity_regression_model.to_csv(os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_sitreps.csv'))

In [35]:
incident_summary_df.to_csv(os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_summaryreps.csv'))

In [36]:
from sklearn.model_selection import train_test_split
X = incident_summary_df
y = incident_summary_df[hazards]
Xtrain_val, Xtest, ytrain_val, ytest = train_test_split(X,y, random_state=42, test_size=0.1, shuffle=True)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain_val, ytrain_val, random_state=42, test_size=(0.1/0.9), shuffle=True)
train_ids = Xtrain['INCIDENT_ID'].unique()
sitreps_train = severity_regression_model.loc[severity_regression_model['INCIDENT_ID'].isin(train_ids)].reset_index(drop=True)
test_ids = Xtest['INCIDENT_ID'].unique()
sitreps_test = severity_regression_model.loc[severity_regression_model['INCIDENT_ID'].isin(test_ids)].reset_index(drop=True)
val_ids = Xval['INCIDENT_ID'].unique()
sitreps_val = severity_regression_model.loc[severity_regression_model['INCIDENT_ID'].isin(val_ids)].reset_index(drop=True)

In [37]:
#checking that training, testing, and validation set all have unique ides
train_ids = set(train_ids)
test_ids = set(test_ids)
val_ids = set(val_ids)
if (train_ids & test_ids) or (train_ids & val_ids) or (test_ids & val_ids):
    print("error: same incident in multiple sets")
else:
    print("OK")

OK


In [38]:
#minmax scaling of contiuous
cont_vals = ["TOTAL_PERSONNEL", "TOTAL_AERIAL", "PCT_CONTAINED_COMPLETED",
              "ACRES",  "WF_FSR", "INJURIES", "FATALITIES", "EST_IM_COST_TO_DATE", "STR_DAMAGED",
              "STR_DESTROYED", "NEW_ACRES","NUM_REPORTS", "DAYS_BURING", 'Current_total_Injuries',
             'Current_total_Structures_Damages','Current_total_Structures_Destroyed', 'Current_total_Fatalities']
minMaxScaler_cls=MinMaxScaler()
minMaxScaler_cls.fit(sitreps_train[cont_vals])
sitreps_train[cont_vals]=minMaxScaler_cls.transform(sitreps_train[cont_vals])
sitreps_val[cont_vals]=minMaxScaler_cls.transform(sitreps_val[cont_vals])
sitreps_test[cont_vals]=minMaxScaler_cls.transform(sitreps_test[cont_vals])
#one hot encoding for any categorical (region, incident management type)
regions = pd.get_dummies(sitreps_train.Incident_region, prefix='Incident_region')
sitreps_train = pd.concat([sitreps_train, regions], axis=1)
regions = pd.get_dummies(sitreps_val.Incident_region, prefix='Incident_region')
sitreps_val = pd.concat([sitreps_val, regions], axis=1)
regions = pd.get_dummies(sitreps_test.Incident_region, prefix='Incident_region')
sitreps_test = pd.concat([sitreps_test, regions], axis=1)
#inc type
inc_type = pd.get_dummies(sitreps_train.INC_MGMT_ORG_ABBREV, prefix='INC_MGMT_ORG_ABBREV')
sitreps_train = pd.concat([sitreps_train, inc_type], axis=1)
inc_type = pd.get_dummies(sitreps_val.INC_MGMT_ORG_ABBREV, prefix='INC_MGMT_ORG_ABBREV')
sitreps_val = pd.concat([sitreps_val, inc_type], axis=1)
inc_type = pd.get_dummies(sitreps_test.INC_MGMT_ORG_ABBREV, prefix='INC_MGMT_ORG_ABBREV')
sitreps_test = pd.concat([sitreps_test, inc_type], axis=1)

In [39]:
sitreps_train.to_csv(os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_sitreps_train.csv'))
sitreps_test.to_csv(os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_sitreps_test.csv'))
sitreps_val.to_csv(os.path.join(os.path.dirname(os.getcwd()),'data','ICS_predictive_sitreps_val.csv'))